# [RunnableMap](https://python.langchain.com/docs/expression_language/how_to/map)并行化

 RunnableParallel（又名RunnableMap）可以很容易地并行执行多个 Runnables，并将这些 Runnable 的输出作为映射返回。

In [2]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

from langchain.globals import set_debug
set_debug(False) 

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel

model = ChatOpenAI()
joke_chain = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model
poem_chain = (
    ChatPromptTemplate.from_template("write a 2-line poem about {topic}") | model
)

map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)
joke_chain.invoke({"topic": "bear"})


可以看到，map_chain的运行时间和前两者相近，这说明map_chain里的两个流水线是并行运行的。

## 统一各组件输入输出格式

在`LangChain`的流水线`(pipeline)`中，我们会有多个组件串联执行。每个组件可能期望不同格式的输入或产生不同格式的输出。为了桥接组件之间的格式差异，我们可以在它们之间插入RunnableMap来转换数据格式，这样外部用户只需要关心自己的输入输出即可。下面是一些映射的常见使用场景:

- 将标记(token)流转换为字符串或其他自定义数据类型
- 解析自然语言并提取结构化数据,如实体、关系等
- 将无结构文本整理为结构化的数据,如JSON
- 调整数据维度,如展平、旋转等
- 过滤或清理无用的数据
  
下面的示例中，`prompt`组件需要一个包含"`context`"和"`question`"键的字典作为输入。而用户只输入了一个问题字符串。于是使用了一个`RunnableMap`，通过检索器`retriever`获取上下文，并用`RunnablePassthrough`传递用户输入的问题字符串，这样就得到了prompt组件需要的输入格式。



In [5]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
# 定义了一个检索器 retriever,可以从文本中检索上下文
retriever = vectorstore.as_retriever()  
# template 定义了一个带占位符的prompt模板，所以prompt组件需要一个包含context和question键的字典作为输入。
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("where did harrison work?")


'Harrison worked at Kensho.'

上面代码中，我们定义了`retrieval_chain`流水线:

- 首先是一个字典，通过`retriever`获得`context`，并使用`RunnablePassthrough`直接传递用户输入作为`question`。
- 传入`prompt`组件，其中字典被自动转换成了`RunnableMap`。
- 最后通过解析器处理`prompt`的输出。


在这个示例代码中，`RunnableMap`的功能是通过传递`RunnablePassthrough`来实现的。我们使用了一个字典包装检索器`retriever`和`RunnablePassthrough`，分别提供了`prompt`需要的"`context`"和"`question`"。


In [ ]:
{"context": retriever, "question": RunnablePassthrough()}

`RunnablePassthrough`可以自动捕获并传递用户的原始输入，作为"`question`"的值。而在连接时，由于有`Runnable`的参与，类型转换是自动完成的，不需要做明确的`RunnableMap`包装。而如果是要明确写出来，可以是：



In [ ]:
from langchain_core.runnables import RunnableMap

retrieval_map = RunnableMap(
    {"context": retriever, "question": RunnablePassthrough()}
)
retrieval_chain = (
    retrieval_map
    | prompt
    | model
    | StrOutputParser()  
)


 这样代码中就会更明确一些，表示我们这里准备的是一个可运行的 map，需要自动进行类型转换，以匹配后续组件的输入。

  可见，`RunnableMap`不仅可以并行组合多个流水线，还能用来格式转换，这加强了它在构建流水线上的灵活性。